In [1]:
# Instead of the terminal mode, you can also try ChatGPT in a python script
# Here is an example.
#%% 
# Importing libs
import os
from frchat.frchatbot import FRChatBot
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv("dev.env"))  # read local .env file

In [2]:
# Providing a context to start a conversation
messages = [
    {'role':'system', 'content':'你是一个机器人用户助手,帮助用户控制一个6关节机器人运动,你需要组合使用一些机器人控制函数来完成特定任务,只使用用户提供的函数,不要自行引入其他第三方库(如RoboDK)'},
    {'role':'user', 'content':'你好，我们开始编写机器人控制指令吧'},
    {'role':'assistant', 'content':'好的，请告诉我能够使用哪些机器人控制函数'},
]

In [3]:
# Defining a ChatBot and claiming the CONFIG_DIR
frcb = FRChatBot(messages, temperature=0.1)
config_dir = os.path.join(os.path.dirname(os.path.split(__file__)[0]), "config/fr_robot/")

In [4]:
# Providing some functions
file = config_dir+'robot_movement/jog.yaml'
output = frcb.generate_function_list(file)
prompt = f"""
    以下是可使用的函数:
    ```{output}```
"""
frcb.chat(prompt)

USER: 
    以下是可使用的函数:
    ```[['StartJOG(ref,nb,dir,vel,acc,max_dis)', "功能:'jog点动'", "参数:'ref:0-关节点动,2-基坐标系点动,4-工具坐标系点动,8-工件坐标系点动(int型变量), nb:1-1关节(x轴),2-2关节(y轴),3-3关节(z轴),4-4关节(rx),5-5关节(ry),6-6关节(rz)(int型变量), dir:0-负方向,1-正方向(int型变量), vel:速度百分比，[0~100](float型变量), acc:加速度百分比，[0~100](float型变量), max_dis:单次点动最大角度/距离,单位°或mm(float型变量)'", "返回值:'成功：[0],失败：[errcode]'"], ['StopJOG(ref)', "功能:'jog点动减速停止'", "参数:'ref:1-关节点动停止,3-基坐标系点动停止,5-工具坐标系点动停止,9-工件坐标系点动停止(int型变量)'", "返回值:'成功：[0],失败：[errcode]'"], ['ImmStopJOG()', "功能:'jog点动立即停止'", "参数:'n/a:n/a(n/a型变量)'", "返回值:'成功：[0],失败：[errcode]'"]]```

FR: 好的，我们可以根据这些函数来编写机器人控制指令。请问需要实现哪些具体的任务呢？


In [5]:
# Describing a task
prompt = f"""
    任务1:请开始点动,1关节旋转,单次最大旋转5度,旋转到正30度,可使用循环
    请生成该任务的python指令
"""
frcb.chat(prompt)

USER: 
    任务1:请开始点动,1关节旋转,单次最大旋转5度,旋转到正30度,可使用循环
    请生成该任务的python指令

FR: 好的，以下是实现该任务的Python指令：

```python
import time

# 定义机器人控制函数
def StartJOG(ref, nb, dir, vel, acc, max_dis):
    # 该函数用于控制机器人点动运动
    pass

def StopJOG(ref):
    # 该函数用于控制机器人点动运动减速停止
    pass

def ImmStopJOG():
    # 该函数用于控制机器人点动运动立即停止
    pass

# 任务1：1关节旋转到正30度
joint_num = 1  # 旋转的关节编号
max_dis = 5  # 单次最大旋转角度
target_angle = 30  # 目标旋转角度
ref = 0  # 关节点动
dir = 1  # 正方向
vel = 50  # 速度百分比
acc = 50  # 加速度百分比

current_angle = 0  # 当前旋转角度
while current_angle < target_angle:
    StartJOG(ref, joint_num, dir, vel, acc, max_dis)
    time.sleep(0.1)  # 等待机器人旋转到目标角度
    current_angle += max_dis
StopJOG(ref)
```

请注意，以上代码中的机器人控制函数只是示例，需要根据实际情况进行修改。同时，需要根据具体机器人的控制方式和通信协议来编写相应的控制指令。


In [6]:
# Providing more functions
file = config_dir+'robot_statusinquiry.yaml'
output = frcb.generate_function_list(file)
prompt = f"""
    新增可使用的函数:
    ```{output}```
    请重写之前的指令，可使用关节角度获取函数获取关节位置
"""
frcb.chat(prompt)

USER: 
    新增可使用的函数:
    ```[['GetRobotInstallAngle()', "功能:'获取机器人安装角度'", "参数:'n/a:n/a(n/a型变量)'", "返回值:'成功：[0,yangle,zangle],yangle-倾斜角，zangle-旋转角;失败：[errcode]'"], ['GetSysVarValue(id)', "功能:'获取系统变量值'", "参数:'id:系统变量编号，范围[1~20](int型变量)'", "返回值:'成功：[0,var_value],失败：[errcode]'"], ['GetActualJointPosDegree(flag)', "功能:'获取关节当前位置(角度)'", "参数:'flag:0-阻塞，1-非阻塞(int型变量)'", "返回值:'成功：[0,joint_pos],joint_pos=[j1,j2,j3,j4,j5,j6];失败：[errcode]'"], ['GetActualJointPosRadian(flag)', "功能:'获取关节当前位置(弧度)'", "参数:'flag:0-阻塞，1-非阻塞(int型变量)'", "返回值:'成功：[0,joint_pos],joint_pos=[j1,j2,j3,j4,j5,j6];失败：[errcode]'"], ['GetActualTCPPose(flag)', "功能:'获取当前工具位姿'", "参数:'flag:0-阻塞，1-非阻塞(int型变量)'", "返回值:'成功：[0,tcp_pose],tcp_pose=[x,y,z,rx,ry,rz];失败：[errcode]'"], ['GetActualTCPNum(flag)', "功能:'获取当前工具坐标系编号'", "参数:'flag:0-阻塞，1-非阻塞(int型变量)'", "返回值:'成功：[0,tool_id],失败：[errcode]'"], ['GetActualWObjNum(flag)', "功能:'获取当前工件坐标系编号'", "参数:'flag:0-阻塞，1-非阻塞(int型变量)'", "返回值:'成功：[0,wobj_id],失败：[errcode]'"], ['GetActualToolFlangePose(flag)', 

In [7]:
# Expanding the task
prompt = f"""
    任务2:关节配置1-[0,0,0,0,0,0],关节配置2-[30,-60,90,20,-10,30], \
    关节点动从配置1运动到配置2,请帮我生成python指令,注意不要使用用户未提供的函数
"""
frcb.chat(prompt)

USER: 
    任务2:关节配置1-[0,0,0,0,0,0],关节配置2-[30,-60,90,20,-10,30],     关节点动从配置1运动到配置2,请帮我生成python指令,注意不要使用用户未提供的函数

FR: 好的，以下是实现该任务的Python指令：

```python
import time

# 定义机器人控制函数
def StartJOG(ref, nb, dir, vel, acc, max_dis):
    # 该函数用于控制机器人点动运动
    pass

def StopJOG(ref):
    # 该函数用于控制机器人点动运动减速停止
    pass

def ImmStopJOG():
    # 该函数用于控制机器人点动运动立即停止
    pass

def GetActualJointPosDegree(flag):
    # 该函数用于获取当前关节角度
    pass

# 任务2：关节点动从关节配置1运动到关节配置2
joint_config1 = [0, 0, 0, 0, 0, 0]  # 关节配置1
joint_config2 = [30, -60, 90, 20, -10, 30]  # 关节配置2
ref = 0  # 关节点动
dir = 1  # 正方向
vel = 50  # 速度百分比
acc = 50  # 加速度百分比

# 将机器人移动到关节配置1
for i in range(6):
    joint_num = i + 1
    max_dis = abs(joint_config1[i] - GetActualJointPosDegree(0)[i])
    StartJOG(ref, joint_num, dir, vel, acc, max_dis)
    time.sleep(0.1)  # 等待机器人移动到目标位置
StopJOG(ref)

# 将机器人移动到关节配置2
for i in range(6):
    joint_num = i + 1
    max_dis = abs(joint_config2[i] - GetActualJointPosDegree(0)[i])
    StartJOG(ref, joint_num, dir, v